In [39]:
import os
import re
import csv
import codecs
import pandas as pd
from datetime import datetime

In [40]:
def to_date(date_reference):
    """
    Funcao que recebe o ano (YYYY), ano e mes (YYYYmm) ou ano, mes e dia (YYYYmmdd)
    no tipo string e o transforma no tipo date.
    
    Args:
        date_reference(str): Ano, ano e mes ou ano, mes e dia.
            Ex.: '2011' ou '201105' ou '20110526'
    
    Returns:
        date: Data de referencia no formato YYYY-mm-dd.
            Ex.: '2011-05-26'
    """
    
    # Define o ano, mes e dia de referencia do arquivo
    if len(date_reference) == 4:
        year = date_reference[:4]
        month = '01'
        day = '01'
    elif len(date_reference) == 6:
        year = date_reference[:4]
        month = date_reference[4:6]
        day = '01'
    elif len(date_reference) == 8:
        year = date_reference[:4]
        month = date_reference[4:6]
        day = date_reference[6:]
    
    # Define a data de referencia do arquivo
    date = pd.to_datetime(year+'-'+month+'-'+day)
    
    return date

In [41]:
def file_references(data_file):
    """
    Funcao que recebe o nome do arquivo original e extrai o assunto e
    data de referencia.
    
    Args:
        data_file(str): Nome do arquivo original.
            Ex.: '201101_GastosDiretos.csv'
    
    Returns:
        date: Data de referencia do arquivo.
            Ex.: '2011-01-01'
        string: Assunto de referencia do arquivo.
            Ex.: 'GastosDiretos'
    """
    # Expressao Regular para:
    # Identificar a data de referencia do arquivo
    date_reference_re = re.compile(r'_A_\d+-\d+-(\d+)')
    
    # Identifica a data de referencia do arquivo
    date_reference = re.search(date_reference_re, data_file).group(1)
        
    # Define a data de referencia do arquivo
    date = to_date(date_reference)
    
    return date

In [42]:
def clean_string(text):
    """
    Funcao que recebe um texto (string) e o padroniza.
    Os passos são:
    1. Insere o termo SIGILOSO nos registros protegidos por sigilo.
    2. Insere o termo INDISPONIVEL nos registros com detalhamento nao disponivel.
    3. Remove espacos brancos extras no final dos registros.
    
    Args:
        text(str): Texto de um registro do arquivo original.
            Ex.: 'Texto com espacos extras    '
    
    Returns:
        str: Texto padronizado.
            Ex.: 'Texto com espacos extras'
    """
    # Expressao regular que identifica campos protegidos por sigilo
    confidential_re = re.compile(r'protegidas por sigilo')
    unavailable_re = re.compile(r'Detalhamento das informa')
    
    if pd.isnull(text):
        return text
    else:
        c = confidential_re.search(text)
        i = unavailable_re.search(text)
        if c:
            return u'Sigiloso'
        elif i:
            return u'Indisponivel'
        else:
            return text.strip()

In [43]:
def clean_float(text):
    """
    Funcao que recebe um campo de valor numerico
    no tipo string e o tranforma no tipo float.
    Os passos sao:
    1. Remove qualquer caractere nao numerico;
    2. Substitui a virgula por ponto;
    3. Transforma no tipo de dado float;

    Args:
        text(str): Valor numerico em tipo string.
            Ex.: '1.500,70'

    Returns:
        float: Valor numerico em tipo float.
            Ex.: 1500.70
    """
    # Expressao regular que identifica caracteres não numericos
    replace_comma_re = re.compile(r'\,')
    
    if pd.isnull(text):
        return text
    elif text == '':
        return ''
    elif text == 'F':
        return ''
    else:
        dot_number = re.sub(replace_comma_re, '.', text)
        return float(dot_number)

In [44]:
def load_data_frame(in_fields, in_data_path):
    """
    Funcao que recebe o caminho completo de um arquivo de dados ja codificado
    em UTF-8, prepara-o em um data frame e renomeia as colunas.
    
    Args:
        in_fields(array): Lista com os nomes das colunas para o data frame.
            Ex.: ['cd_campo', 'nm_campo', 'dt_campo', 'vl_campo']
        in_data_path(str): Caminho completo de acesso ao arquivo de dados.
            Ex.: '..\\data\\encoded\\201301_GastosDiretos.csv'
        
    Returns:
        dataframe: Data frame do arquivo original com as colunas renomeadas.
    """
    # Le o arquivo original em um dataframe
    df = pd.read_csv(in_data_path
                    ,sep=';'
                    #,quotechar = '\"'
                    ,quoting = csv.QUOTE_NONE
                    ,low_memory = False
                    ,encoding = 'utf-8'
                    ,dtype = 'object')
    
    #Renomeia as colunas
    df.columns = in_fields
    
    return df

In [45]:
def data_frame_to_csv(df, out_data_path):
    """
    Funcao que recebe o caminho completo para escrita do dataframe e o
    transforma em arquivo CSV, codificado em UTF-8 e separado por tab.
    
    Args:
        df(dataframe): Dataframe com os campos padronizados.
        out_data_path(str): Caminho completo para escrita do dataframe em csv.
            Ex.: '..\\data\\padronized\\Favorecidos_GastosDiretos_2013-01-01.csv'
    """
    df.to_csv(path_or_buf = out_data_path
             ,index = False
             ,sep = '\t'
             ,encoding = 'utf-8')

In [46]:
def select_fields(df, fields_list):
    """
    Funcao que recebe um data frame e retorna uma copia com apenas as
    colunas informadas.
    
    Args:
        df(dataframe): Data frame enviado para recorte.
        fields_list(array): Lista de colunas do dataframe para selecao.
        
    Returns:
        dataframe: Copia do dataframe apenas com as colunas informadas.
    """
    return df.loc[:,fields_list].copy()

In [47]:
def add_data_source_fields(df, **kwargs):
    """
    Funcao que adiciona ao final do dataframe uma sequencia de campos
    referentes a fonte dos dados que o originou.
    
    Args:
        df(dataframe): Dataframe com os campos padronizados.
        date_reference(date): Data de referencia do arquivo original.
            Ex.: '2013-01-01'
        data_source(str): Nome da fonte do arquivo original.
            Ex.: 'Portal da Transparência'
        data_file(str): Nome do arquivo original.
            Ex,: 'GastosDiretos_201301.csv'
        
    Returns:
        dataframe: Dataframe com os campos referentes a 
            fonte de dados adicionado.
    """
    df['dt_referencia'] = kwargs['date_reference']
    df['nm_fonte_dados'] = kwargs['data_source']
    df['nm_arquivo_dados'] = kwargs['data_file'] 
    
    return df

In [48]:
def standardize_historico(**kwargs):
    """
    Recebe a localizacao fisica do arquivo organizado, renomeia as colunas, padroniza os tipos 
    de dados (texto, numerico e data) e cria um novo arquivo padronizado no local informado.
    
    Args:
        data_file(str): Nome do arquivo original.
            Ex.: '201001_CPGF.csv'
        date_reference(date): Data de referencia do arquivo original.
            Ex.: '2010-01-01'
        in_data_path(str): Caminho completo para acesso ao arquivo original.
            Ex.: '..\data\03-organized\201001_CPGF.csv'
        out_data_path(str): Caminho completo onde deve ser criado o(s) novo(s)
            arquivo(s) com os dados padronizados.
            Ex.: '..\data\04-standardized\201001_CPGF.csv'
    """
        
    # Nome das colunas que irao substituir o nome das colunas originais
    kwargs['in_fields'] = ['sg_regiao'
                          ,'sg_uf'
                          ,'nm_estacao'
                          ,'cd_estacao_wmo'
                          ,'vl_latitude'
                          ,'vl_longitude'
                          ,'vl_altitude'
                          ,'dt_fundacao'
                          ,'dt_medicao'
                          ,'hr_medicao'
                          ,'vl_precipitacao_mm'
                          ,'vl_pressao_atmosferica_mb'
                          ,'vl_pressao_atmosferica_mb_max_aut'
                          ,'vl_pressao_atmosferica_mb_min_aut'
                          ,'vl_radiacao_global_kj_m2'
                          ,'vl_temperatura_ar_c_bulbo_seco'
                          ,'vl_temperatura_orvalho_c'
                          ,'vl_temperatura_c_max_aut'
                          ,'vl_temperatura_c_min_aut'
                          ,'vl_temperatura_orvalho_c_max_aut'
                          ,'vl_temperatura_orvalho_c_min_aut'
                          ,'vl_umidade_relativa_max_aut'
                          ,'vl_umidade_relativa_min_aut'
                          ,'vl_umidade_relativa'
                          ,'vl_vento_direcao_horaria_gr'
                          ,'vl_vento_rajada_max_ms'
                          ,'vl_vento_velocidade_ms'
                          ,'vl_none']
    
    # Prepara o CSV original em um dataframe e renomeia as colunas 
    df = load_data_frame(kwargs['in_fields'], kwargs['in_data_path'])
    
    # Padroniza todos os campos do dataframe
    df = df.applymap(lambda x: clean_string(x))
    
    # Padroniza os campos de data e numericos
    df['ts_medicao'] = df['dt_medicao'] + ' ' + df['hr_medicao']
    df['vl_latitude'] = df['vl_latitude'].apply(clean_float)
    df['vl_longitude'] = df['vl_longitude'].apply(clean_float)
    df['vl_altitude'] = df['vl_altitude'].apply(clean_float)
    df['vl_precipitacao_mm'] = df['vl_precipitacao_mm'].apply(clean_float)
    df['vl_pressao_atmosferica_mb'] = df['vl_pressao_atmosferica_mb'].apply(clean_float)
    df['vl_pressao_atmosferica_mb_max_aut'] = df['vl_pressao_atmosferica_mb_max_aut'].apply(clean_float)
    df['vl_pressao_atmosferica_mb_min_aut'] = df['vl_pressao_atmosferica_mb_min_aut'].apply(clean_float)
    df['vl_radiacao_global_kj_m2'] = df['vl_radiacao_global_kj_m2'].apply(clean_float)
    df['vl_temperatura_ar_c_bulbo_seco'] = df['vl_temperatura_ar_c_bulbo_seco'].apply(clean_float)
    df['vl_temperatura_orvalho_c'] = df['vl_temperatura_orvalho_c'].apply(clean_float)
    df['vl_temperatura_c_max_aut'] = df['vl_temperatura_c_max_aut'].apply(clean_float)
    df['vl_temperatura_c_min_aut'] = df['vl_temperatura_c_min_aut'].apply(clean_float)
    df['vl_temperatura_orvalho_c_max_aut'] = df['vl_temperatura_orvalho_c_max_aut'].apply(clean_float)
    df['vl_temperatura_orvalho_c_min_aut'] = df['vl_temperatura_orvalho_c_min_aut'].apply(clean_float)
    df['vl_umidade_relativa_max_aut'] = df['vl_umidade_relativa_max_aut'].apply(clean_float)
    df['vl_umidade_relativa_min_aut'] = df['vl_umidade_relativa_min_aut'].apply(clean_float)
    df['vl_umidade_relativa'] = df['vl_umidade_relativa'].apply(clean_float)
    df['vl_vento_direcao_horaria_gr'] = df['vl_vento_direcao_horaria_gr'].apply(clean_float)
    df['vl_vento_rajada_max_ms'] = df['vl_vento_rajada_max_ms'].apply(clean_float)
    df['vl_vento_velocidade_ms'] = df['vl_vento_velocidade_ms'].apply(clean_float)
    
    # Inclui os campos de informacao da fonte dos dados
    df = add_data_source_fields(df, **kwargs)
    
    # Nome das colunas que irao substituir o nome das colunas originais
    kwargs['out_fields'] = ['sg_regiao'
                           ,'sg_uf'
                           ,'nm_estacao'
                           ,'cd_estacao_wmo'
                           ,'vl_latitude'
                           ,'vl_longitude'
                           ,'vl_altitude'
                           ,'dt_fundacao'
                           ,'ts_medicao'
                           ,'vl_precipitacao_mm'
                           ,'vl_pressao_atmosferica_mb'
                           ,'vl_pressao_atmosferica_mb_max_aut'
                           ,'vl_pressao_atmosferica_mb_min_aut'
                           ,'vl_radiacao_global_kj_m2'
                           ,'vl_temperatura_ar_c_bulbo_seco'
                           ,'vl_temperatura_orvalho_c'
                           ,'vl_temperatura_c_max_aut'
                           ,'vl_temperatura_c_min_aut'
                           ,'vl_temperatura_orvalho_c_max_aut'
                           ,'vl_temperatura_orvalho_c_min_aut'
                           ,'vl_umidade_relativa_max_aut'
                           ,'vl_umidade_relativa_min_aut'
                           ,'vl_umidade_relativa'
                           ,'vl_vento_direcao_horaria_gr'
                           ,'vl_vento_rajada_max_ms'
                           ,'vl_vento_velocidade_ms'
                           ,'dt_referencia'
                           ,'nm_fonte_dados'
                           ,'nm_arquivo_dados']
    
    # Prepara um data frame com apenas as colunas do arquivo de saida
    sub_df = select_fields(df, kwargs['out_fields'])
    
    # Salva o arquivo de saida no local informado
    data_frame_to_csv(sub_df, kwargs['out_data_path'])

    # Log: Mensagem de fim do processo
    print(str(datetime.now()) + ': Arquivo ' + kwargs['data_file'] + ' padronizado.')

In [49]:
def standardize_files():
    """
    Acessa os arquivos da pasta 03-organized, passa pelo processo de padronizacao
    e armazena-os na pasta 04-standardized.
    """

    # Prepara o dicionario de variaveis (kwargs = keyworded arguments)
    kwargs = {}
    
    # Nome da fonte dos dados
    kwargs['data_source'] = u'Instituto Nacional de Meteorologia'
    
    # Diretorio de armazenamento dos arquivos originais
    kwargs['in_data_dir'] = '..\\data\\03-organized'
    
    # Diretorio de armazenamento dos arquivos tratados
    kwargs['out_data_dir'] = '..\\data\\04-standardized'
    
    # Lista dos arquivos organizados 
    kwargs['data_files'] = os.listdir(kwargs['in_data_dir'])
    
    # Log: Mensagem de inicio do processo
    print(str(datetime.now()) + ': Padronizacao dos arquivos iniciada.')
    
    # Para cada arquivo na lista de arquivos organizados
    for file in kwargs['data_files']:
        
        # Define o nome do arquivo
        kwargs['data_file'] = file

        # Define o caminho completo de acesso ao arquivo original
        kwargs['in_data_path'] = os.path.join(kwargs['in_data_dir'], kwargs['data_file'])

        # Define o caminho completo de armazenamento do arquivo padronizado
        kwargs['out_data_path'] = os.path.join(kwargs['out_data_dir'], kwargs['data_file'])
        
        # Define data e assunto de referencia do arquivo
        kwargs['date_reference'] = file_references(kwargs['data_file'])

        # Padroniza os arquivos
        standardize_historico(**kwargs)    
      
    # Log: Mensagem de finalizacao do processo
    print(str(datetime.now()) + ': Padronizacao dos arquivos finalizada.')
    

In [50]:
def main():
    """
    Padroniza os dados meteorologicos das estacoes automaticas. 
    Origem: Instituto Nacional de Meteorologia 
    """
    
    ### Padroniza os arquivos
    standardize_files()

if __name__ == '__main__':
    main()

2021-05-27 16:08:06.037406: Padronizacao dos arquivos iniciada.
2021-05-27 16:08:07.243784: Arquivo INMET_CO_DF_A001_BRASILIA_01-01-2001_A_31-12-2001.CSV padronizado.
2021-05-27 16:08:08.384589: Arquivo INMET_CO_DF_A001_BRASILIA_01-01-2002_A_31-12-2002.CSV padronizado.
2021-05-27 16:08:09.560498: Arquivo INMET_CO_DF_A001_BRASILIA_01-01-2003_A_31-12-2003.CSV padronizado.
2021-05-27 16:08:10.711948: Arquivo INMET_CO_DF_A001_BRASILIA_01-01-2004_A_31-12-2004.CSV padronizado.
2021-05-27 16:08:11.874103: Arquivo INMET_CO_DF_A001_BRASILIA_01-01-2005_A_31-12-2005.CSV padronizado.
2021-05-27 16:08:13.016590: Arquivo INMET_CO_DF_A001_BRASILIA_01-01-2006_A_31-12-2006.CSV padronizado.
2021-05-27 16:08:14.417009: Arquivo INMET_CO_DF_A001_BRASILIA_01-01-2007_A_31-12-2007.CSV padronizado.
2021-05-27 16:08:15.578145: Arquivo INMET_CO_DF_A001_BRASILIA_01-01-2008_A_31-12-2008.CSV padronizado.
2021-05-27 16:08:16.736869: Arquivo INMET_CO_DF_A001_BRASILIA_01-01-2009_A_31-12-2009.CSV padronizado.
2021-05-2

2021-05-27 16:09:34.942086: Arquivo INMET_CO_GO_A003_MORRINHOS_01-01-2011_A_31-12-2011.CSV padronizado.
2021-05-27 16:09:36.083371: Arquivo INMET_CO_GO_A003_MORRINHOS_01-01-2012_A_31-12-2012.CSV padronizado.
2021-05-27 16:09:37.322595: Arquivo INMET_CO_GO_A003_MORRINHOS_01-01-2013_A_31-12-2013.CSV padronizado.
2021-05-27 16:09:38.469043: Arquivo INMET_CO_GO_A003_MORRINHOS_01-01-2014_A_31-12-2014.CSV padronizado.
2021-05-27 16:09:39.611614: Arquivo INMET_CO_GO_A003_MORRINHOS_01-01-2015_A_31-12-2015.CSV padronizado.
2021-05-27 16:09:40.783390: Arquivo INMET_CO_GO_A003_MORRINHOS_01-01-2016_A_31-12-2016.CSV padronizado.
2021-05-27 16:09:41.941931: Arquivo INMET_CO_GO_A003_MORRINHOS_01-01-2017_A_31-12-2017.CSV padronizado.
2021-05-27 16:09:43.085174: Arquivo INMET_CO_GO_A003_MORRINHOS_01-01-2018_A_31-12-2018.CSV padronizado.
2021-05-27 16:09:44.261753: Arquivo INMET_CO_GO_A003_MORRINHOS_01-01-2019_A_31-12-2019.CSV padronizado.
2021-05-27 16:09:45.407087: Arquivo INMET_CO_GO_A003_MORRINHOS_0

2021-05-27 16:11:04.285504: Arquivo INMET_CO_GO_A015_ITAPACI_01-01-2016_A_31-12-2016.CSV padronizado.
2021-05-27 16:11:05.433630: Arquivo INMET_CO_GO_A015_ITAPACI_01-01-2017_A_31-12-2017.CSV padronizado.
2021-05-27 16:11:06.580588: Arquivo INMET_CO_GO_A015_ITAPACI_01-01-2018_A_31-12-2018.CSV padronizado.
2021-05-27 16:11:07.698525: Arquivo INMET_CO_GO_A015_ITAPACI_01-01-2019_A_31-12-2019.CSV padronizado.
2021-05-27 16:11:08.844667: Arquivo INMET_CO_GO_A015_ITAPACI_01-01-2020_A_31-12-2020.CSV padronizado.
2021-05-27 16:11:10.092555: Arquivo INMET_CO_GO_A015_ITAPACI_03-02-2007_A_31-12-2007.CSV padronizado.
2021-05-27 16:11:11.256037: Arquivo INMET_CO_GO_A016_JATAI_01-01-2008_A_31-12-2008.CSV padronizado.
2021-05-27 16:11:12.545742: Arquivo INMET_CO_GO_A016_JATAI_01-01-2009_A_31-12-2009.CSV padronizado.
2021-05-27 16:11:13.695700: Arquivo INMET_CO_GO_A016_JATAI_01-01-2010_A_31-12-2010.CSV padronizado.
2021-05-27 16:11:14.870157: Arquivo INMET_CO_GO_A016_JATAI_01-01-2011_A_31-12-2011.CSV p

2021-05-27 16:12:34.640636: Arquivo INMET_CO_GO_A025_RIO VERDE_01-01-2011_A_31-12-2011.CSV padronizado.
2021-05-27 16:12:35.795979: Arquivo INMET_CO_GO_A025_RIO VERDE_01-01-2012_A_31-12-2012.CSV padronizado.
2021-05-27 16:12:36.944962: Arquivo INMET_CO_GO_A025_RIO VERDE_01-01-2013_A_31-12-2013.CSV padronizado.
2021-05-27 16:12:38.094314: Arquivo INMET_CO_GO_A025_RIO VERDE_01-01-2014_A_31-12-2014.CSV padronizado.
2021-05-27 16:12:39.244227: Arquivo INMET_CO_GO_A025_RIO VERDE_01-01-2015_A_31-12-2015.CSV padronizado.
2021-05-27 16:12:40.397212: Arquivo INMET_CO_GO_A025_RIO VERDE_01-01-2016_A_31-12-2016.CSV padronizado.
2021-05-27 16:12:41.548449: Arquivo INMET_CO_GO_A025_RIO VERDE_01-01-2017_A_31-12-2017.CSV padronizado.
2021-05-27 16:12:42.698593: Arquivo INMET_CO_GO_A025_RIO VERDE_01-01-2018_A_31-12-2018.CSV padronizado.
2021-05-27 16:12:43.810455: Arquivo INMET_CO_GO_A025_RIO VERDE_01-01-2019_A_31-12-2019.CSV padronizado.
2021-05-27 16:12:44.733756: Arquivo INMET_CO_GO_A025_RIO VERDE_0

2021-05-27 16:14:00.847706: Arquivo INMET_CO_GO_A033_PIRES DO RIO_01-01-2018_A_31-12-2018.CSV padronizado.
2021-05-27 16:14:01.985034: Arquivo INMET_CO_GO_A033_PIRES DO RIO_01-01-2019_A_31-12-2019.CSV padronizado.
2021-05-27 16:14:03.135134: Arquivo INMET_CO_GO_A033_PIRES DO RIO_01-01-2020_A_31-12-2020.CSV padronizado.
2021-05-27 16:14:03.415116: Arquivo INMET_CO_GO_A033_PIRES DO RIO_12-10-2007_A_31-12-2007.CSV padronizado.
2021-05-27 16:14:04.553070: Arquivo INMET_CO_GO_A034_CATALAO_01-01-2009_A_31-12-2009.CSV padronizado.
2021-05-27 16:14:05.693480: Arquivo INMET_CO_GO_A034_CATALAO_01-01-2010_A_31-12-2010.CSV padronizado.
2021-05-27 16:14:06.850198: Arquivo INMET_CO_GO_A034_CATALAO_01-01-2011_A_31-12-2011.CSV padronizado.
2021-05-27 16:14:08.011640: Arquivo INMET_CO_GO_A034_CATALAO_01-01-2012_A_31-12-2012.CSV padronizado.
2021-05-27 16:14:09.156413: Arquivo INMET_CO_GO_A034_CATALAO_01-01-2013_A_31-12-2013.CSV padronizado.
2021-05-27 16:14:10.294875: Arquivo INMET_CO_GO_A034_CATALAO_0

...

2021-05-27 18:14:04.630278: Arquivo INMET_S_RS_A881_DOM PEDRITO_01-01-2020_A_31-12-2020.CSV padronizado.
2021-05-27 18:14:05.473985: Arquivo INMET_S_RS_A881_DOM PEDRITO_23-04-2010_A_31-12-2010.CSV padronizado.
2021-05-27 18:14:06.639617: Arquivo INMET_S_RS_A882_TEUTONIA_01-01-2013_A_31-12-2013.CSV padronizado.
2021-05-27 18:14:07.817236: Arquivo INMET_S_RS_A882_TEUTONIA_01-01-2014_A_31-12-2014.CSV padronizado.
2021-05-27 18:14:08.980567: Arquivo INMET_S_RS_A882_TEUTONIA_01-01-2015_A_31-12-2015.CSV padronizado.
2021-05-27 18:14:10.155174: Arquivo INMET_S_RS_A882_TEUTONIA_01-01-2016_A_31-12-2016.CSV padronizado.
2021-05-27 18:14:11.343920: Arquivo INMET_S_RS_A882_TEUTONIA_01-01-2017_A_31-12-2017.CSV padronizado.
2021-05-27 18:14:12.487776: Arquivo INMET_S_RS_A882_TEUTONIA_01-01-2018_A_31-12-2018.CSV padronizado.
2021-05-27 18:14:13.737171: Arquivo INMET_S_RS_A882_TEUTONIA_01-01-2019_A_31-12-2019.CSV padronizado.
2021-05-27 18:14:14.909727: Arquivo INMET_S_RS_A882_TEUTONIA_01-01-2020_A_31

2021-05-27 18:15:27.367386: Arquivo INMET_S_SC_A806_FLORIANOPOLIS_01-01-2013_A_31-12-2013.CSV padronizado.
2021-05-27 18:15:28.558483: Arquivo INMET_S_SC_A806_FLORIANOPOLIS_01-01-2014_A_31-12-2014.CSV padronizado.
2021-05-27 18:15:29.710524: Arquivo INMET_S_SC_A806_FLORIANOPOLIS_01-01-2015_A_31-12-2015.CSV padronizado.
2021-05-27 18:15:30.870800: Arquivo INMET_S_SC_A806_FLORIANOPOLIS_01-01-2016_A_31-12-2016.CSV padronizado.
2021-05-27 18:15:32.021004: Arquivo INMET_S_SC_A806_FLORIANOPOLIS_01-01-2017_A_31-12-2017.CSV padronizado.
2021-05-27 18:15:33.200542: Arquivo INMET_S_SC_A806_FLORIANOPOLIS_01-01-2018_A_31-12-2018.CSV padronizado.
2021-05-27 18:15:34.373976: Arquivo INMET_S_SC_A806_FLORIANOPOLIS_01-01-2019_A_31-12-2019.CSV padronizado.
2021-05-27 18:15:35.563332: Arquivo INMET_S_SC_A806_FLORIANOPOLIS_01-01-2020_A_31-12-2020.CSV padronizado.
2021-05-27 18:15:36.766302: Arquivo INMET_S_SC_A806_FLORIANOPOLIS_22-01-2003_A_31-12-2003.CSV padronizado.
2021-05-27 18:15:37.947532: Arquivo I

2021-05-27 18:16:57.345481: Arquivo INMET_S_SC_A845_BOM JARDIM DA SERRA - MORRO DA IGREJA_01-01-2010_A_31-12-2010.CSV padronizado.
2021-05-27 18:16:58.512603: Arquivo INMET_S_SC_A845_BOM JARDIM DA SERRA - MORRO DA IGREJA_01-01-2011_A_31-12-2011.CSV padronizado.
2021-05-27 18:16:59.688886: Arquivo INMET_S_SC_A845_BOM JARDIM DA SERRA - MORRO DA IGREJA_01-01-2012_A_31-12-2012.CSV padronizado.
2021-05-27 18:17:00.884109: Arquivo INMET_S_SC_A845_BOM JARDIM DA SERRA - MORRO DA IGREJA_01-01-2013_A_31-12-2013.CSV padronizado.
2021-05-27 18:17:02.040600: Arquivo INMET_S_SC_A845_BOM JARDIM DA SERRA - MORRO DA IGREJA_01-01-2014_A_31-12-2014.CSV padronizado.
2021-05-27 18:17:03.231698: Arquivo INMET_S_SC_A845_BOM JARDIM DA SERRA - MORRO DA IGREJA_01-01-2015_A_31-12-2015.CSV padronizado.
2021-05-27 18:17:04.416888: Arquivo INMET_S_SC_A845_BOM JARDIM DA SERRA - MORRO DA IGREJA_01-01-2016_A_31-12-2016.CSV padronizado.
2021-05-27 18:17:05.601049: Arquivo INMET_S_SC_A845_BOM JARDIM DA SERRA - MORRO DA 

2021-05-27 18:18:23.998006: Arquivo INMET_S_SC_A859_CACADOR_01-01-2019_A_31-12-2019.CSV padronizado.
2021-05-27 18:18:25.132997: Arquivo INMET_S_SC_A859_CACADOR_01-01-2020_A_31-12-2020.CSV padronizado.
2021-05-27 18:18:26.060455: Arquivo INMET_S_SC_A859_CACADOR_19-03-2008_A_31-12-2008.CSV padronizado.
2021-05-27 18:18:27.224965: Arquivo INMET_S_SC_A860_CURITIBANOS_01-01-2009_A_31-12-2009.CSV padronizado.
2021-05-27 18:18:28.390763: Arquivo INMET_S_SC_A860_CURITIBANOS_01-01-2010_A_31-12-2010.CSV padronizado.
2021-05-27 18:18:29.567325: Arquivo INMET_S_SC_A860_CURITIBANOS_01-01-2011_A_31-12-2011.CSV padronizado.
2021-05-27 18:18:30.925642: Arquivo INMET_S_SC_A860_CURITIBANOS_01-01-2012_A_31-12-2012.CSV padronizado.
2021-05-27 18:18:32.064497: Arquivo INMET_S_SC_A860_CURITIBANOS_01-01-2013_A_31-12-2013.CSV padronizado.
2021-05-27 18:18:33.232415: Arquivo INMET_S_SC_A860_CURITIBANOS_01-01-2014_A_31-12-2014.CSV padronizado.
2021-05-27 18:18:34.438447: Arquivo INMET_S_SC_A860_CURITIBANOS_01-

2021-05-27 18:19:54.643340: Arquivo INMET_S_SC_A866_Laguna  - Farol de Santa Marta_01-01-2013_A_31-12-2013.CSV padronizado.
2021-05-27 18:19:55.830920: Arquivo INMET_S_SC_A866_Laguna  - Farol de Santa Marta_01-01-2014_A_31-12-2014.CSV padronizado.
2021-05-27 18:19:57.050063: Arquivo INMET_S_SC_A866_Laguna  - Farol de Santa Marta_01-01-2015_A_31-12-2015.CSV padronizado.
2021-05-27 18:19:58.955536: Arquivo INMET_S_SC_A866_Laguna  - Farol de Santa Marta_01-01-2016_A_31-12-2016.CSV padronizado.
2021-05-27 18:20:00.119754: Arquivo INMET_S_SC_A866_Laguna  - Farol de Santa Marta_01-01-2017_A_31-12-2017.CSV padronizado.
2021-05-27 18:20:01.432993: Arquivo INMET_S_SC_A866_Laguna  - Farol de Santa Marta_01-01-2018_A_31-12-2018.CSV padronizado.
2021-05-27 18:20:02.598502: Arquivo INMET_S_SC_A866_Laguna  - Farol de Santa Marta_01-01-2019_A_31-12-2019.CSV padronizado.
2021-05-27 18:20:03.756276: Arquivo INMET_S_SC_A866_Laguna  - Farol de Santa Marta_01-01-2020_A_31-12-2020.CSV padronizado.
2021-05-